In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from IPython import display
from IPython.core.display import display as disp, HTML
disp(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Setup the sensor geometry
# All units in mm

sensor = {
    'px': .1e-3,   #Pixel size
    'npix':[1024, 1024]   #Number of pixels
    }

sensor['fov'] = [sensor['npix'][0]*sensor['px'], sensor['npix'][1]*sensor['px']]


In [ ]:
def genz(ii, rhomax):
    # Generates Zernike polynomial function handles
    #
    # Inputs
    # ii : Zernike index
    # rhomax : max lateral coordinate, for normalization
    #
    # outputs:
    # Z : surface height map
    # fx: x gradient
    # fy: y gradient
    # These are handles meant to be evaluated on cartesian coordinates
    
    if ii == 1:
        Z = lambda x, y : np.ones_like(x)
        fx = lambda x, y : 0
        fy = lambda x, y : 0
    elif ii == 2:
        Z = lambda x, y : x/rhomax
        fx = lambda x, y : 1/rhomax
        fy = lambda x, y : 0
    elif ii == 3:
        Z = lambda x, y : y/rhomax
        fx = lambda x, y : 0
        fy = lambda x, y : 1/rhomax    
    elif ii == 4:
        Z = lambda x, y : (x/rhomax)**2 - (y/rhomax)**2
        fx = lambda x, y : 2*x/(rhomax**2)
        fy = lambda x, y : -2*y/(rhomax**2)
    elif ii == 5:
        Z = lambda x, y : -1 + 2*(x/rhomax)**2 + 2*(y/rhomax)**2
        fx = lambda x, y : 4*x/(rhomax**2)
        fy = lambda x, y : 4*y/(rhomax**2)
    elif ii == 6:
        Z = lambda x, y : 2*x*y/(rhomax**2)
        fx = lambda x, y : 2*y/(rhomax**2)
        fy = lambda x, y : 2*x/(rhomax**2)
    elif ii == 7:
        Z = lambda x, y : (x/rhomax)**3 - 3*(x/rhomax)*(y/rhomax)**2
        fx = lambda x, y : 3*x**2/(rhomax**3) - 3*y**2/(rhomax**3)
        fy = lambda x, y : 6*x*y/(rhomax**3)
    elif ii == 8:
        Z = lambda x, y : 3*(x/rhomax)**3 + 3*(x/rhomax)*(y/rhomax)**2 - 2*x/rhomax
        fx = lambda x, y : 9*x**2/(rhomax**3) + 3*y**2/(rhomax**3) - 2/rhomax
        fy = lambda x, y : 6*x*y/(rhomax**3)
    elif ii == 9:
        Z = lambda x, y : 3*(y/rhomax)**3 + 3*(x/rhomax)**2*(y/rhomax) - 2*y/rhomax
        fx = lambda x, y : 6*x*y/(rhomax**3)
        fy = lambda x, y : 9*y**2/(rhomax**3) + 3*x**2/(rhomax**3) - 2/rhomax
    elif ii == 10:
        Z = lambda x, y : -(y/rhomax)**3 + 3*(x/rhomax)**2*y/rhomax
        fx = lambda x, y : 6*x*y/(rhomax**3)
        fy = lambda x, y : -3*y**2/(rhomax**3) + 3*x**2/(rhomax**3)
    return Z, fx, fy

In [ ]:
def genW2(zlist, coeffs, rhomax):
    # Creates function handles to generate wavefronts as linear combinations 
    # of zernike polynomials
    #
    # Inputs
    # zlist : list of zernike indices
    # coeffs : corresponding coefficients
    # rhomax : maximum physical coordinate of the pupil, used to normalize
    #
    # Outputs
    # W : Wavefront func handle, can be evaluated at 2D coordinates
    # fx: x gradient handle
    # fy: y gradient handle
    
    zern = []    
    for ii in range(len(zlist)):
        zern.append(genz(zlist[ii],rhomax))

    W = lambda x,y:sum([zern[ii][0](x,y)*coeffs[ii] for ii in range(len(zlist))])
    fx = lambda x,y:sum([zern[ii][1](x,y)*coeffs[ii] for ii in range(len(zlist))])
    fy = lambda x,y:sum([zern[ii][2](x,y)*coeffs[ii] for ii in range(len(zlist))])
    return W, fx, fy

In [ ]:
def genW(zlist,coeffs,rhomax):
    # Old. don't use.
    W = lambda x,y:sum([genz(zlist[ii],rhomax)[0](x,y)*coeffs[ii] for ii in range(len(zlist))])
    fx = lambda x,y:sum([genz(zlist[ii],rhomax)[1](x,y)*coeffs[ii] for ii in range(len(zlist))])
    fy = lambda x,y:sum([genz(zlist[ii],rhomax)[2](x,y)*coeffs[ii] for ii in range(len(zlist))])
    return W, fx, fy

In [ ]:
def genSin(wx,wy,amp):
    # Generates a sinusoidal phase map for modeling gratings.
    # inputs
    # wx : x-direction spatial frequency (in radians)
    # wy : y-direction spatial frequency
    # amp : amplitude of sinusoid
    #
    # Outputs
    # W : Wavefront func handle, can be evaluated at 2D coordinates
    # fx: x gradient handle
    # fy: y gradient handle
    
    W = lambda x,y:amp*(np.sin(wx*x) + np.sin(wy*y))
    fx = lambda x,y:amp*wx*np.cos(wx*x)
    fy = lambda x,y:amp*wy*np.cos(wy*y)
    return W, fx, fy

In [ ]:
# Test the generation of wavefronts using zernikes
rx = np.linspace(-sensor['fov'][1]/2,sensor['fov'][1]/2,sensor['npix'][1]);
ry = np.linspace(-sensor['fov'][0]/2,sensor['fov'][0]/2,sensor['npix'][0]);

[Rx,Ry] = np.meshgrid(rx,ry)

zlist = 1+np.r_[1:10]
zco = np.random.randn(len(zlist))



In [ ]:
def genSph(efl):
    # Create function handles for spherical wavefront
    # and derivatives, fx and fy.
    # The curvature is designed so that it focuses at a deistance efl
    #
    # Inputs
    # 3fl : effective focal length of the surface (from vertex)
    #
    # Outputs
    # W : Wavefront func handle, can be evaluated at 2D coordinates
    # fx: x gradient handle
    # fy: y gradient handle
    
    Wsph = lambda x,y : efl - efl*np.real(np.sqrt(1 - (x/efl)**2 - (y/efl)**2 + 0j))
    #fx = lambda x,y: x/np.sqrt(1 - (x/efl)**2 - (y/efl)**2)/efl
    #fy = lambda x,y: y/np.sqrt(1 - (x/efl)**2 - (y/efl)**2)/efl
    fx = lambda x,y: x/(efl * np.sqrt(1 - (y/efl)**2 - (x/efl)**2))
    fy = lambda x,y: y/(efl * np.sqrt(1 - (y/efl)**2 - (x/efl)**2))
    return Wsph, fx, fy


In [ ]:
# Construct a wavefront as a surface

zlist = np.r_[1:11]    #List of zernikes to use
print(zlist)
zco = .001*np.random.randn(len(zlist))   #Random coefficients

# Zero out tilt
zco[0] = 0    
zco[1] = .00
zco[2] = .00
zco[zlist==5] = 0.

# Sinusoidal wavefront constants
wx = 1000    #x frequency
wy = 500    #y frequency
amp = .000001   #Amplitude of oscillaions


In [ ]:
# Define the pupil of the system

lam = 550e-6   #Wavelength

pupil = {
    'ca':[.5, .5],   # Clear aperture in x and y
    'efl' : 1,   #Focal lenght, used for spherical component calculation.
    't': 1,    #Distance to sensor
    'ap_type' : 'circ'    #Aperture shape. 'circ' or 'rect'
}

rhomax = pupil['ca'][0]/2

#Generate wavefront functions and add to pupil         
W = genW2(zlist, zco, rhomax)
pupil['W_aber'] = W[0]
pupil['fx_aber'] = W[1]
pupil['fy_aber'] = W[2]

W_sph = genSph(pupil['efl']);
pupil['W_sph'] = W_sph[0]
pupil['fx_sph'] = W_sph[1]
pupil['fy_sph'] = W_sph[2]

W_sin = genSin(wx,wy,amp)
pupil['W_sin'] = W_sin[0]
pupil['fx_sin'] = W_sin[1]
pupil['fy_sin'] = W_sin[2]

In [ ]:
W = genW2(zlist, zco, rhomax)
pupil['W_aber'] = W[0]
pupil['fx_aber'] = W[1]
pupil['fy_aber'] = W[2]

plt.figure(figsize=(20,20))
plt.subplot(3,3,1)
plt.imshow(pupil['W_sph'](Rx,Ry))
plt.title('Spherical component')
plt.colorbar()

plt.subplot(3,3,2)
plt.imshow(pupil['fx_sph'](Rx,Ry))
plt.title('Spherical x-derivative')
plt.colorbar()

plt.subplot(3,3,3)
plt.imshow(pupil['fy_sph'](Rx,Ry))
plt.title('Spherical y-derivative')
plt.colorbar()

plt.subplot(3,3,4)
plt.imshow(pupil['W_sin'](Rx,Ry))
plt.title('Sinusoidal component')
plt.colorbar()

plt.subplot(3,3,5)
plt.imshow(pupil['fx_sin'](Rx,Ry))
plt.title('Sinusoidal x-derivative')
plt.colorbar()

plt.subplot(3,3,6)
plt.imshow(pupil['fy_sin'](Rx,Ry))
plt.title('Sinusoidal y-derivative')
plt.colorbar()

plt.subplot(3,3,7)
plt.imshow(pupil['W_aber'](Rx,Ry))
plt.title('Zernike component')
plt.colorbar()

plt.subplot(3,3,8)
plt.imshow(pupil['fx_aber'](Rx,Ry))
plt.title('Zernike x-derivative')
plt.colorbar()

plt.subplot(3,3,9)
plt.imshow(pupil['fy_aber'](Rx,Ry))
plt.title('Zernike y-derivative')
plt.colorbar()

In [ ]:
def trace_ray(rhox, rhoy, pupil,debug=False):
    # Function that computes a ray's position and OPD at a flat plane,
    # with the ray originating from an arbitrary wavefront.
    # The ray direction and origin in z will be determined by the sum 
    # of wavefront maps defined in the pupil.
    #
    # Inputs
    # rhox : ray x coordinate
    # rhoy : ray y coordinate
    # pupil : dictionary containing pupil information.
    # debug : print stuff if true
    #
    # Outputs
    # xo and yo : ray output (x,y) coordinates
    # opl : ray path length
    # W : total wavefront (useful for analysis later)
    # fx and fy : ray direction
    # K : x direction cosine
    # L : y direction cosine
    # M : z direction cosine
    
    
    # Generate wavefront and gradients:
    W = pupil['W_sph'](rhox,rhoy) + pupil['W_aber'](rhox,rhoy) + pupil['W_sin'](rhox,rhoy)
    fx = pupil['fx_sph'](rhox,rhoy) + pupil['fx_aber'](rhox,rhoy) + pupil['fx_sin'](rhox,rhoy)
    fy = pupil['fy_sph'](rhox,rhoy) + pupil['fy_aber'](rhox,rhoy) + pupil['fy_sin'](rhox,rhoy)
    
    # z0 is the total propagton distance
    z0 = W - pupil['t']
    nlength = np.sqrt(1 + fx**2 + fy**2)   #Length of the wavefront normal vector
    
    # x,y, and z direction cosines, respectively
    K = -fx/nlength 
    L = -fy/nlength
    M = 1/nlength
    
    # Total propagation length along ray
    opl = -z0/M

    # Change in x and y ray coordinates
    dx = K*opl
    dy = L*opl
    
    # Output x and y coordinates
    
    yo = rhoy + dy
    xo = rhox + dx;
    
    #debugging stuff. 
    if debug:
        print('(rhox, rhoy):',rhox,rhoy)
        print('(Fx,Fy)',fx,fy)
        print('(dx, dy, dz)=',dx,dy,dz)        
        print('(xo,yo,zo)',xo,yo,z0)
        print('(K,L,M)=',K,L,M)
        print('Delta:', Delta)
        print('\n')

    

    return xo, yo, opl, W, fx, fy, K, L, M

In [ ]:
# Rays-only loop
Nrays = 2
xo = []
yo = []
opl = []
for k in range(Nrays):
    
    # Pick a random location on the surface
    if pupil['ap_type'] == 'circ':
        thx = np.random.rand(1)*2*np.pi
        rho = pupil['ca'][0]/2*np.sqrt(np.random.rand(1))
        rhox = rho*np.sin(thx)
        rhoy = rho*np.cos(thx)
    elif pupil['ap_type'] == 'rect':
        rhox = pupil['ca'][1]*(np.random.rand(1) - 0.5)
        rhoy = pupil['ca'][0]*(np.random.rand(1) - 0.5)

    

    # Launch a ray toward the sensor
    rays_out = trace_ray(rhox, rhoy, pupil,debug=False)
    xo.append(rays_out[0])
    yo.append(rays_out[1])
    opl.append(rays_out[2])


In [ ]:
#a,b,c,Wt,Fxt,Fyt = trace_ray(Rx,Ry,pupil)

In [ ]:
# Scatter plot ray output positoins
f = plt.figure(figsize=(15,15))
ax = plt.gca()
ax.set_facecolor((0,0,0))
plt.scatter(xo,yo,s=50,c=[[1, 1, 1]],marker='.',alpha=.1)
ax.axis('equal')

In [ ]:

#Integrate rays into pixels using 2D histogram

yoarr = np.asarray(yo)[:,0]
xoarr = np.asarray(xo)[:,0]
binedgex = np.r_[:sensor['npix'][1]+1]*sensor['px']
binedgex -= np.mean(binedgex)
binedgey = np.r_[:sensor['npix'][0]+1]*sensor['px']
binedgey -= np.mean(binedgey)
H,xe, ye = np.histogram2d(xoarr,yoarr,
                          bins=[sensor['npix'][1],sensor['npix'][0]],
                          range=[[-sensor['fov'][1]/2,sensor['fov'][1]/2],[-sensor['fov'][0]/2,sensor['fov'][0]/2]])


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))
figextent = [-sensor['fov'][1]/2,sensor['fov'][1]/2,-sensor['fov'][0]/2,sensor['fov'][0]/2]
ax[1].imshow(H.transpose()[::-1], extent=figextent,interpolation='none',cmap='gray')
ax[1].set_title('pixels')
ax[0].scatter(xo,yo,s=.01,c=[[1, 1, 1]],marker='.',alpha=1)
ax[0].set_facecolor((0,0,0))

ax[0].set(xlim=(-sensor['fov'][1]/2,sensor['fov'][1]/2),ylim=(-sensor['fov'][0]/2,sensor['fov'][0]/2))
ax[0].set_aspect('equal', adjustable='box')
ax[0].set_title('scatter plot with transparency')

In [ ]:
def ray2wave(xo,yo,fx,fy,opl,lamb, Sx, Sy):
    # Generate plane wave based on ray data
    # 
    # Inputs: 
    # xo,yo : x,y location at plane
    # fx,fy : x,y derivatives (slope of ray)
    # opl : Ray optical path length
    # lamb : Wavelength
    # Sx, Sy : meshgrided coordinates of sensor pixels
    #
    # Outputs
    # U : Complex field of plane wave evaluated on sensor grid
    k = 2*np.pi/lamb
    opd = k*opl
    nnorm = np.sqrt(fx**2 + fy**2 + 1)
    kx = -fx*k/nnorm
    ky = -fy*k/nnorm
#    kz = np.sqrt(k**2 - kx**2 - ky **2)

    
    U = np.exp(1j*(-kx*(Sx-xo) - ky*(Sy-yo) - opd))
    return U


In [ ]:
def drawnow(fig):
    display.clear_output(wait=True)
    display.display(fig)

In [ ]:
# Plane waves loop
Nrays = 2
xo = []
yo = []
opl = []
fx = []
fy = []
dx = []
dy = []

U_accum = np.zeros(sensor['npix']) + 1j*np.zeros(sensor['npix'])   #This will accumulate the total field
f = plt.figure(figsize=(20,20))   #Figure handle

# Generate sensor pixel grid
sx = np.linspace(-sensor['fov'][1]/2,sensor['fov'][1]/2,sensor['npix'][1])
sy = np.linspace(-sensor['fov'][0]/2,sensor['fov'][0]/2,sensor['npix'][0])
Sx,Sy = np.meshgrid(sx,sy)

# Loop that randomly samples rays
for kk in range(Nrays):
    
    
    # Pick a random location on the surface
    if pupil['ap_type'] == 'circ':
        # Generate rays randomly in polar coordinates
        thx = np.random.rand(1)*2*np.pi     #Pick random ray azimuth
        rho = pupil['ca'][0]/2*np.sqrt(np.random.rand(1))   #Random ray radial location
        
        #Convert to cartesian
        rhox = rho*np.sin(thx)
        rhoy = rho*np.cos(thx)
        
    elif pupil['ap_type'] == 'rect':
        rhox = pupil['ca'][1]*(np.random.rand(1) - 0.5)
        rhoy = pupil['ca'][0]*(np.random.rand(1) - 0.5)

    # Launch a ray toward the sensor
    rays_out = trace_ray(rhox, rhoy, pupil,debug=False)
    # This part "remembers" ray data. Commented out because not needed
#     xo.append(rays_out[0])
#     yo.append(rays_out[1])
#     dx.append(xo[kk] - rhox)
#     dy.append(yo[kk] - rhoy)
#     opl.append(rays_out[2])
#     fx.append(rays_out[4])
#     fy.append(rays_out[5])
    xo_d = rays_out[0]
    yo_d = rays_out[1]
    fx_d = rays_out[4]
    fy_d = rays_out[5]
    opl_d = rays_out[2]
  
    U = ray2wave(xo_d, yo_d, fx_d, fy_d, opl_d, lam, Sx, Sy)
    
    U_accum += U
    if kk%100==0:
        plt.imshow(np.abs(U_accum)**2)#,extent=figextent)
        #plt.imshow(np.real(U))
        plt.title(kk)
        print(kk)
        drawnow(f)
    # Create the corresponding plane wave on the sensor grid

    # Add this plane wave to the existing distribution

    # Repeat

U_out = U_accum



In [ ]:
# Compare to Fresnel propagation
def propagate_field(lam, ps, z, U):
    k = 2*np.pi/lam   
    siz = np.shape(U)
    fx = np.linspace(-1/2/ps,1/2/ps,siz[1])
    fy = np.linspace(-1/2/ps,1/2/ps,siz[0])
    x = np.linspace(-siz[1]/2*ps,siz[1]/2*ps,siz[1])
    y = np.linspace(-siz[0]/2*ps,siz[0]/2*ps,siz[0])
    X,Y = np.meshgrid(x,y)
    Fx,Fy = np.meshgrid(fx,fy)
    Uf = np.fft.fftshift(np.fft.fft2(U))
    Hf = np.exp(1j*2*np.pi*z/lam * np.sqrt(0j+1-(lam*Fx)**2 - (lam*Fy)**2))
    Up = np.fft.ifft2(np.fft.ifftshift(Uf*Hf))
    return Up

In [ ]:
a={'a_thing':12}
b = copy.deepcopy(a)
b['a_thing']=5
a['a_thing']

In [ ]:
sensor_FFT = copy.deepcopy(sensor)
sensor_FFT['npix']=np.array([5112, 5112])
sensor_FFT['fov'] = [sensor_FFT['npix'][0]*sensor_FFT['px'],
                     sensor_FFT['npix'][1]*sensor_FFT['px']]



rx = np.linspace(-sensor_FFT['fov'][1]/2,sensor_FFT['fov'][1]/2,sensor_FFT['npix'][1]);
ry = np.linspace(-sensor_FFT['fov'][0]/2,sensor_FFT['fov'][0]/2,sensor_FFT['npix'][0]);
[Rx,Ry] = np.meshgrid(rx,ry)
ap = np.where(np.sqrt(Rx**2 + Ry**2)<(pupil['ca'][0]/2),1.,0.)
W = pupil['W_sph'](Rx,Ry) + pupil['W_aber'](Rx,Ry) + pupil['W_sin'](Rx,Ry)
lam = 550e-6
U = ap*np.exp(-1j*2*np.pi/lam*W*1)
U_fresnel = propagate_field(lam, sensor['px'], pupil['t'],U)
#U_crop = U_fresnel[]

In [ ]:
scrop = (sensor_FFT['npix'] - np.array(sensor['npix']))//2
print(scrop)
U_crop = U_fresnel[scrop[0]:-scrop[0],scrop[0]:-scrop[0]]

In [ ]:
PSF_huyg = np.abs(U_accum[::-1,:])**2
PSF_fresnel = np.abs(U_crop[::-1,:])**2

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(30,10))
ax[0].imshow(H.transpose()[::-1],extent=figextent,vmax=10)
ax[0].set_title('Rays only')
ax[1].imshow(PSF_huyg,interpolation='none',extent=figextent)
ax[1].set_title('Diffraction from rays (monochromatic)')
ax[2].imshow(PSF_fresnel,interpolation='none',extent=figextent)
ax[2].set_title('Fresnel propagation')